In [17]:
#--This codes is used for Hacker News text mining of the "upvotes" 
#  title messages and giving a preditions based on a linearRegression algorithm--

In [18]:
# read modules
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression

In [19]:
#-- Read the data from sel_hn_stories.csv file from columns 
# submission_time, upvotes, url, headline --
submissions = pd.read_csv("sel_hn_stories.csv")
submissions.columns = ["submission_time", "upvotes", "url", "headline"]
submissions = submissions.dropna()
tokenized_headlines = []
for item in submissions["headline"]:
    tokenized_headlines.append(item.split(" "))

In [20]:
#-- Get rid of the punctuations and cleansing the data --
punctuation = [",", ":", ";", ".", "'", '"', "’", "?", "/", "-", "+", "&", "(", ")"]
clean_tokenized = []
for item in tokenized_headlines:
    tokens = []
    for token in item:
        token = token.lower()
        for punc in punctuation:
            token = token.replace(punc,"")
        tokens.append(token)
    clean_tokenized.append(tokens)

In [23]:
#-- Digitalize the title words and put results in single_tokens and unique_tokens --
unique_tokens = []
single_tokens = []
for tokens in clean_tokenized:
    for token in tokens:
        if token not in single_tokens:
            single_tokens.append(token)
        elif token in single_tokens and token not in unique_tokens:
            unique_tokens.append(token)            
counts = pd.DataFrame(0, index = np.arange(len(clean_tokenized)), columns = unique_tokens)
#-- Generalize the unique tokens matrix
for i, item in enumerate(clean_tokenized):
    for token in item:
        if token in unique_tokens:
            counts.iloc[i][token] += 1
#-- Select the features from its appearing more than 5 times and less than 100 times
world_counts = counts.sum(axis = 0)
counts = counts.loc[:,(world_counts >= 5) & (world_counts <= 100)]

In [27]:
#-- Prepare training group and testing group before the linearRegression prediction --
X_train, X_test, y_train, y_test = train_test_split(counts, submissions["upvotes"], test_size=0.2, random_state=1)
#-- Import the LinearRegression Model --
clf = LinearRegression()
clf.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [28]:
#-- Prediction preform --
predictions = clf.predict(X_test)

In [29]:
#-- Mean squared error (MSE) measurement
mse = sum((predictions -y_test)**2)/len(predictions)

In [30]:
mse

2652.6082512522867